In [2]:
using NamedArrays
Gases = [:gas1, :gas2, :gas3]

min_octane = Dict(zip(Gases,[10,8,6]))
max_sulfer = Dict(zip(Gases,[1,2,1]))
gas_price = Dict(zip(Gases,[70,60,50]))
gas_nom_demand = Dict(zip(Gases,[3000,2000,1000]))

Crudes = [:crude1, :crude2, :crude3]
crude_price = Dict(zip(Crudes,[45,35,25]))
octane = Dict(zip(Crudes,[12,6,8]))
sulfer = Dict(zip(Crudes,[0.5,2.0,3.0]))

max_crude_available = 5000
max_crude_processed = 14000
processing_cost = 4
advertising_inc = 10

using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
set_silent(m)

@variable(m, y[Crudes] >= 0)
@variable(m, z[Gases] >= 0)
@variable(m, x[Crudes,Gases] >= 0)
@variable(m, a[Gases] >= 0)

# Let's make the objective with expressions
@expression(m, revenue, sum(gas_price[j]*z[j] for j in Gases))
@expression(m, crude_cost, sum(crude_price[i]*y[i] for i in Crudes))
@expression(m, production_cost, 4*sum(y[i] for i in Crudes))
@expression(m, advertising_costs, sum(a[j] for j in Gases))
@objective(m, Max, revenue - crude_cost - production_cost - advertising_costs)

# Be sure to define y in terms of x
@constraint(m, [i in Crudes], sum(x[i,j] for j in Gases) == y[i])

# Be sure to define z in terms of x
@constraint(m, [j in Gases], sum(x[i,j] for i in Crudes) == z[j])


# Individual crude maximum 
@constraint(m, [i in Crudes], y[i] <= max_crude_available)

# Total crude processing
@constraint(m, sum(y[i] for i in Crudes) <= max_crude_processed)

# Demand
@constraint(m, [j in Gases], z[j] <= gas_nom_demand[j] + a[j])

# Min average octane
@constraint(m, [j in Gases], sum(octane[i]*x[i,j] for i in Crudes) >= min_octane[j]*z[j])

# Max average sulfer
@constraint(m, [j in Gases], sum(sulfer[i]*x[i,j] for i in Crudes) <= max_sulfer[j]*z[j])

optimize!(m)
println("Max profit: ", objective_value(m))
for j in Gases
    if value(a[j]) > 0.001
        println("Advertise for ", j, " : ", value(a[j]))
    end
end






Max profit: 310000.0
Advertise for gas2 : 9000.0
